In [2]:
import dotenv
from openai import AzureOpenAI
from swarm import Swarm, Agent
import requests

In [5]:
def get_weather(latitude: float, longitude: float) -> str:
    """
    使用Open-Meteo API获取给定坐标的当前天气。

    参数:
    latitude (float): 纬度
    longitude (float): 经度

    返回:
    str: 包含当前温度和风速的天气信息JSON字符串
    """
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        current = data['current']
        return str(current)  # 返回JSON字符串
    else:
        return '{"error": "无法获取天气数据"}'

def get_coordinates(location: str) -> str:
    """
    获取给定位置名称的坐标。

    参数:
    location (str): 位置名称

    返回:
    str: 包含纬度和经度的JSON字符串
    """
    coordinates = {
        "纽约": (40.7128, -74.0060),
        "伦敦": (51.5074, -0.1278),
        "东京": (35.6762, 139.6503),
        "巴黎": (48.8566, 2.3522),
        "柏林": (52.5200, 13.4050)
    }
    lat, lon = coordinates.get(location.lower(), (0, 0))
    return f'{{"latitude": {lat}, "longitude": {lon}}}'

In [3]:
dotenv.load_dotenv()
Azure_client = AzureOpenAI()
client = Swarm(client=Azure_client)

In [7]:
weather_agent = Agent(
    name="天气助手",
    instructions="""
    你是一个由帮助的天气助手。当被问到特定位置的天气时：
    1. 使用get_coordinates函数获取该位置的坐标。
    2. 使用get_weather函数获取天气数据。
    3. 解析返回的JSON数据，提供一个友好的回复，包含天气信息。
    如果无法识别该位置，礼貌地通知用户并建议他们尝试一个主要城市。
    """,
    functions=[get_coordinates, get_weather],
)

def run_weather_query(query: str) -> str:
    """
    运行天气查询并返回结果

    参数：
    query (str): 用户的天气查询

    返回:
    str: Agent的响应
    """
    messages = [{"role":'user', "content": query}]
    response = client.run(agent=weather_agent, messages=messages)
    return response.messages[-1]["content"]

In [8]:
print(run_weather_query("纽约的天气怎么样？"))

纽约现在的天气温度为-3.3°C，风速为7.8m/s。注意保暖哦！如果有进一步的计划，注意查看最新的天气预报以做好准备。


In [9]:
print(run_weather_query("乌兰察布的天气怎么样？"))

我无法识别乌兰察布这个位置。请您尝试输入一个主要城市的名称，以便我为您提供准确的天气信息。
